In [1]:
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime
import pytz
import math

# ================= 🔧 KONFIGURASI =================
CRED_PATH = "/work/staklimjerukagung-firebase-admin.json"
DB_URL = "https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/"
DB_NODE_PATH = "/auto_weather_stat/id-05/data"

# RENTANG WAKTU TARGET (WIB)
START_TIME_STR = "2025-11-29 03:09:00"
END_TIME_STR   = "2025-11-29 07:46:00"

# DATA YANG WAJIB ADA (Jika salah satu hilang/null, key akan dihapus)
REQUIRED_FIELDS = ['temperature', 'humidity', 'dew']

# ==================================================

if not firebase_admin._apps:
    cred = credentials.Certificate(CRED_PATH)
    firebase_admin.initialize_app(cred, {'databaseURL': DB_URL})

def is_incomplete_or_null(data_dict):
    """
    Cek apakah data lengkap dan valid.
    Return True jika data RUSAK (harus dihapus).
    """
    if not isinstance(data_dict, dict):
        return True # Datanya bukan dictionary, hapus aja

    for field in REQUIRED_FIELDS:
        # 1. Cek apakah field-nya ADA?
        if field not in data_dict:
            return True # Data pincang (misal: ada temp, ga ada hum), HAPUS.
        
        val = data_dict[field]

        # 2. Cek apakah nilainya None / Null
        if val is None:
            return True
            
        # 3. Cek NaN (Float)
        if isinstance(val, float) and math.isnan(val):
            return True
            
        # 4. Cek NaN (String "NaN")
        if isinstance(val, str) and val.lower() == 'nan':
            return True
            
    return False # Data Sehat

def delete_incomplete_rows():
    ref = db.reference(DB_NODE_PATH)
    
    # 1. Konversi Waktu
    wib = pytz.timezone('Asia/Jakarta')
    start_dt = wib.localize(datetime.strptime(START_TIME_STR, "%Y-%m-%d %H:%M:%S"))
    end_dt   = wib.localize(datetime.strptime(END_TIME_STR, "%Y-%m-%d %H:%M:%S"))
    
    start_key = str(int(start_dt.timestamp()))
    end_key   = str(int(end_dt.timestamp()))
    
    print(f"🎯 Target Rentang Waktu: {START_TIME_STR} s.d {END_TIME_STR}")
    
    # 2. Ambil Data
    print("⏳ Mengunduh data...")
    snapshot = ref.order_by_key().start_at(start_key).end_at(end_key).get()
    
    if not snapshot:
        print("❌ Tidak ada data.")
        return

    # 3. Filter Data Rusak
    keys_to_destroy = []
    
    for key, val in snapshot.items():
        if is_incomplete_or_null(val):
            keys_to_destroy.append(key)
            
    if not keys_to_destroy:
        print("✅ Data Sehat Semua! Tidak ada yang bolong/null.")
        return

    print(f"\n🚨 Ditemukan {len(keys_to_destroy)} KEY Timestamp yang datanya tidak lengkap/null.")
    print("   Contoh Key yang akan DIHAPUS TOTAL:")
    for k in keys_to_destroy[:5]: 
        print(f"   - {k} (Isinya: {snapshot[k]})")
    
    # 4. Eksekusi
    confirm = input("\nKetik 'HANCURKAN' untuk menghapus key tersebut: ")
    if confirm != 'HANCURKAN':
        print("Dibatalkan.")
        return

    print("\n🔥 Mulai Menghapus Key...")
    count = 0
    
    for key in keys_to_destroy:
        try:
            # Hapus SATU KEY (Timestamp) beserta isinya
            ref.child(key).delete()
            print(f"   🗑️ Musnahkan: {key}")
            count += 1
        except Exception as e:
            print(f"   ❌ Gagal: {e}")

    print(f"✨ Selesai! {count} baris data berhasil dibersihkan.")

if __name__ == "__main__":
    delete_incomplete_rows()

🎯 Target Rentang Waktu: 2025-11-29 03:09:00 s.d 2025-11-29 07:46:00
⏳ Mengunduh data...

🚨 Ditemukan 271 KEY Timestamp yang datanya tidak lengkap/null.
   Contoh Key yang akan DIHAPUS TOTAL:
   - 1764360632 (Isinya: {'pressure': 1010.04, 'rainfall': 1.6764, 'rainrate': 0, 'timestamp': 1764360632, 'volt': 3.9625})
   - 1764360692 (Isinya: {'pressure': 1010.02, 'rainfall': 1.6764, 'rainrate': 0, 'timestamp': 1764360692, 'volt': 3.9525})
   - 1764360812 (Isinya: {'pressure': 1010.07, 'rainfall': 1.6764, 'rainrate': 0, 'timestamp': 1764360812, 'volt': 3.9475})
   - 1764360992 (Isinya: {'pressure': 1010.16, 'rainfall': 1.6764, 'rainrate': 0, 'timestamp': 1764360992, 'volt': 3.93875})
   - 1764361052 (Isinya: {'pressure': 1010.15, 'rainfall': 1.6764, 'rainrate': 0, 'timestamp': 1764361052, 'volt': 3.95125})

🔥 Mulai Menghapus Key...
   🗑️ Musnahkan: 1764360632
   🗑️ Musnahkan: 1764360692
   🗑️ Musnahkan: 1764360812
   🗑️ Musnahkan: 1764360992
   🗑️ Musnahkan: 1764361052
   🗑️ Musnahkan: 1764

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>